# Inspect data

## sparsity of neural data

In [ ]:
dec.binned_spikes_df.shape

In [ ]:
# inspect neural data

bins = dec.binned_spikes_df

# Calculate percentage of non-zero rows for each column
non_zero_percentages = (bins != 0).mean() * 100

# Create a DataFrame with the results
non_zero_df = pd.DataFrame({
    'Column': non_zero_percentages.index,
    'Percent_Non_Zero': non_zero_percentages.values
})

# Sort by percentage in descending order
non_zero_df = non_zero_df.sort_values('Percent_Non_Zero', ascending=False)

print("Percentage of non-zero values in each column:")
print(non_zero_df)


In [ ]:
bins.drop(columns='bin').mean(axis=1).describe()

# plot the percentile of values of mean firing rates across neurons at each time bin
mean_rates = bins.drop(columns='bin').mean(axis=1)

# Calculate percentiles from 0 to 100
percentiles = np.arange(0, 101, 1)
percentile_values = np.percentile(mean_rates, percentiles)

# Create plot
plt.figure(figsize=(6, 4))
plt.plot(percentiles, percentile_values)
plt.xlabel('Percentile')
plt.ylabel('Mean Firing Rate')
plt.title('Distribution of Mean Firing Rates Across Neurons')
plt.grid(True)
plt.show()


## multicollinearity

### y var (behavioral)

In [ ]:
y_var_vif = drop_high_vif_vars.get_vif_df(dec.y_var)
print(y_var_vif.head(8))

# calculate the correlation coefficient among the columns with VIF > 5
# specific_columns = vif_df[vif_df['vif'] > 5].feature.values
specific_columns = y_var_vif.feature.values[:10]
corr_coeff = dec.y_var[specific_columns].corr()
#plt.figure(figsize = (6, 6))
plt.figure(figsize = (8, 6))
sns.heatmap(corr_coeff, cmap='coolwarm', annot=True, linewidths=1, vmin=-1)
plt.show()

In [ ]:
# Try y_var_reduced

y_var_vif = drop_high_vif_vars.get_vif_df(dec.y_var_reduced)
print(y_var_vif.head(8))

# calculate the correlation coefficient among the columns with VIF > 5
# specific_columns = vif_df[vif_df['vif'] > 5].feature.values
specific_columns = y_var_vif.feature.values[:10]
corr_coeff = dec.y_var[specific_columns].corr()
#plt.figure(figsize = (6, 6))
plt.figure(figsize = (8, 6))
sns.heatmap(corr_coeff, cmap='coolwarm', annot=True, linewidths=1, vmin=-1)
plt.show()

### check target_rel_x and y
(The look correct after checking)

In [ ]:
pursuit_sub = dec.pursuit_data.loc[dec.pursuit_data['target_index']==65].copy()
pursuit_sub['target_angle_deg'] = pursuit_sub['target_angle'] * 180/pi 

In [ ]:
pursuit_sub[['point_index', 'target_angle_deg', 'target_distance', 'target_rel_x', 'target_rel_y']]

# Reducing columns in lags: experiment

## check for contributions

In [ ]:
df = dec.y_var_lags_reduced.copy()
# for target_feature in vif_df[vif_df['vif'] > 5].feature.values:
for index, row in vif_df.iterrows():
    target_feature = row.feature
    print(f'\n\n {target_feature}: VIF = {row.vif}')
    contributions = drop_high_vif_vars.check_vif_contribution(df, target_feature)

## check subset's vif

### use y_var_lags

In [ ]:
columns = [col for col in dec.y_var_lags.columns if ('LD' in col) or (
                'RD' in col) or ('gaze_mky_view_angle' in col)]

df_sub = dec.y_var_lags[columns].copy()
print(df_sub.columns)
sub_vif = drop_high_vif_vars.get_vif_df(df_sub)
sub_vif

### use y_var_lags_reduced

In [ ]:
# columns = [col for col in dec.y_var_lags_reduced.columns if ('LD' in col) or (
#                 'RD' in col) or ('gaze_mky_view_angle' in col)]

columns = [col for col in dec.y_var_lags_reduced.columns if ('x_r' in col) or ('y_r' in col)]

df_sub = dec.y_var_lags_reduced[columns].copy()
print(df_sub.columns)
sub_vif = drop_high_vif_vars.get_vif_df(df_sub)
sub_vif

## exp on subsets to reduce

In [ ]:
dec.get_x_and_y_data_for_modeling()

In [ ]:
subset_key_words, all_column_subsets = dec.get_subset_key_words_and_all_column_subsets_for_vif(
            dec.y_var_lags_reduced)
subset_key_words

# Appendix

## check NA

In [ ]:
na_rows, na_cols = general_utils.check_na_in_df(dec.y_var)
duplicate_rows = general_utils.find_duplicate_rows(dec.y_var, column_subset=None)

## test speed of code

In [ ]:
# Run profiler and save output to file
cProfile.run('dec.streamline_making_behav_and_neural_data()', 'profile_output')

# Load stats and sort by total time
p = pstats.Stats('profile_output')
p.strip_dirs().sort_stats('tottime').print_stats(10)

In [ ]:
p.strip_dirs().sort_stats('tottime').print_stats()

## See sizes of biggest variables

### in dec

In [ ]:
from pympler import asizeof

sizes = []
for attr in dir(dec):
    if attr.startswith('__') and attr.endswith('__'):
        continue  # skip dunder attributes
    try:
        val = getattr(dec, attr)
        size = asizeof.asizeof(val)
        sizes.append((attr, size))
    except Exception:
        pass  # ignore any errors

# Sort and display largest attributes in MB
for name, size in sorted(sizes, key=lambda x: x[1], reverse=True):
    print(f"{name}: {size / (1024 * 1024):.2f} MB")


### global

In [ ]:
import types
import warnings
from pympler import asizeof

warnings.filterwarnings("ignore", category=UserWarning)


filtered = {
    k: v for k, v in globals().items()
    if not isinstance(v, types.ModuleType)
}

sizes = []
for name, val in filtered.items():
    try:
        sizes.append((name, asizeof.asizeof(val)))
    except:
        pass

for name, size in sorted(sizes, key=lambda x: x[1], reverse=True)[:20]:
    print(f"{name}: {size / (1024 * 1024):.2f} MB")


## why ratio of bin/target_index approaches constant

In [ ]:
trial_lengths = dec.pursuit_data[['target_index', 'bin']].groupby('target_index').count()
trial_lengths.describe()

In [ ]:
sub = dec.y_var_reduced[['time', 'bin', 'target_index']]
sub['factor'] = dec.y_var_reduced['bin']/dec.y_var_reduced['target_index']
sub

In [ ]:
plt.hist(np.diff(dec.ff_caught_T_sorted), bins=30)
plt.xlabel('Time difference')
plt.ylabel('Count')
plt.title('Distribution of time differences between caught events')
plt.show()


In [ ]:
dec.ff_caught_T_sorted/np.arange(len(dec.ff_caught_T_sorted))

## compared with neural_data_modeling

In [ ]:
dec = neural_vs_behavioral_class.NeuralVsBehavioralClass(raw_data_folder_path=raw_data_folder_path)
dec.streamline_preparing_neural_and_behavioral_data()

In [ ]:
dec.final_behavioral_data

In [ ]:
dec.y_var

In [ ]:
dec.y_var_reduced

In [ ]:
dec.y_var.columns

In [ ]:
dec.y_var_reduced.columns

In [ ]:
[col for col in dec.y_var.columns if col not in dec.y_var_reduced.columns]

In [ ]:
[col for col in dec.y_var_reduced.columns if col not in dec.y_var.columns]

## find out why there are rows of NA in dec.y_var

In [ ]:
na_rows, na_cols = general_utils.check_na_in_df(dec.y_var)
duplicate_rows = general_utils.find_duplicate_rows(dec.y_var, column_subset=None)

In [ ]:
dec.behav_data_by_bin.loc[118189:118195, ['bin', 'time', 'target_rel_x', 'target_rel_y','time_since_target_last_seen', 'target_last_seen_distance']]

## Compare old and new target df

In [ ]:
target_df_ori = pd.read_csv('/Users/dusiyi/Documents/Multifirefly-Project/all_monkey_data/patterns_and_features/monkey_Schro/data_0416/target_df_ori.csv')
df = target_df_ori[['target_index', 'point_index', 'time']].copy()
for col in ['target_distance', 'time_since_target_last_seen']:
    df[f'old_{col}'] = target_df_ori[col]   
    df[f'new_{col}'] = dec.target_df[col]  

df['old_target_last_seen_distance'] = target_df_ori['target_last_seen_distance_frozen']
df['new_target_last_seen_distance'] = dec.target_df['target_last_seen_distance']

df2 = df.loc[10068:]
df2

In [ ]:
df3 = df2[df2['point_index']>= 139910]
df3

## Check gpfa's binned spikes vs my own binned spikes

In [ ]:
dec.prepare_spikes_for_gpfa(align_at_beginning=False)

### take out my own binned spikes

In [ ]:
# Select data for a specific segment
seg_index = 20
p_sub = dec.pursuit_data[dec.pursuit_data['segment'] == seg]

# Get corresponding binned spikes and reassign bin as index
binned_spikes_sub = dec.binned_spikes_df[dec.binned_spikes_df['bin'].isin(p_sub['bin'])].copy()
binned_spikes_sub['bin'] = binned_spikes_sub.index

# Merge in time information from pursuit data
binned_spikes_sub2 = binned_spikes_sub.merge(p_sub[['bin', 'time']], on='bin', how='left')

# Identify the unit with the highest total spike count
cluster_columns = [col for col in binned_spikes_sub2.columns if 'cluster_' in col]
cluster_w_most_spikes = binned_spikes_sub2[cluster_columns].sum(axis=0).idxmax()
cluster_index = cluster_w_most_spikes.split('_')[-1]

# Extract bin, time, and spike count for the most active unit
binned_spikes_sub3 = binned_spikes_sub2[['bin', 'time', cluster_w_most_spikes]].copy()


### get binned spikes (seqs) from gpfa_utils.

In [ ]:
cluster_idx_in_gpfa = np.where(np.sort(dec.spike_segs_df.cluster.unique()) == int(cluster_index))[0][0]
seg = dec.spiketrain_corr_segs[seg_index]
spiketrain = dec.spiketrains[seg_index][cluster_idx_in_gpfa]
seqs = gpfa_util.get_seqs([spiketrain], dec.bin_width_w_unit)

### compare

In [ ]:
trial_length = len(binned_spikes_sub3)
if dec.align_at_beginning:
    binned_spikes_sub3['gpfa'] = seqs[0][1][0][trial_length:]
else:
    binned_spikes_sub3['gpfa'] = seqs[0][1][0][-trial_length:] # when getting latent dimension for neural data, [-trial_length:] was also used
binned_spikes_sub3['same'] = binned_spikes_sub3[cluster_w_most_spikes] == binned_spikes_sub3['gpfa']
binned_spikes_sub3[binned_spikes_sub3['same']!=True]

,bin,time,unit_9,gpfa,same
3,5702,114.05650,4.00000,2.00000,False
10,5709,114.18956,2.00000,1.41421,False
20,5719,114.38853,2.00000,1.41421,False
